In [ ]:
import pandas as pd
import numpy as np
from tabulate import tabulate
import re
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
newyork_population = pd.read_csv('/content/newyorkpop.csv')

newyork_population = newyork_population.drop(columns=['name', 'pop2020', 'year', 'fips', 'state', 'stateCode', 'growthSince2020', 'slug', 'rank', 'densityMi', 'areaMi'])

newyork_population['county'] = newyork_population['county'].str.replace('County', '', regex=False)

newyork_population['pop2024'] = newyork_population['pop2024'].astype(int)

newyork_population['landArea'] = newyork_population['landArea'].astype(int)

newyork_population['density'] = newyork_population['density'].astype(int)

# Renaming the 'county' column in df_washington to match 'County'
newyork_population = newyork_population.rename(columns={'county': 'County'})

newyork_population


,pop2024,County,landArea,density
0,2532919,Kings,69,36708
1,2225834,Queens,109,20420
2,1600359,New York,23,69580
3,1518366,Suffolk,911,1666
4,1378136,Nassau,285,4835
...,...,...,...,...
57,30119,Schoharie,622,48
58,26501,Lewis,1276,20
59,24442,Yates,338,72
60,17405,Schuyler,328,53


In [ ]:
newyork_gdp = pd.read_csv('/content/newyork_gdp.csv', delimiter=';')

newyork_gdp = newyork_gdp.drop(columns=['2020 GDP', '2021 GDP', '2022 GDP', 'Rank in State', '2020 Percent Change', '2021 Percent Change', '2022 Percent Change', '2023 Percent Change'])

newyork_gdp['2023 GDP'] = newyork_gdp['2023 GDP'].str.replace('.', '', regex=False)

# Renaming the 'county' column in df_washington to match 'County'
newyork_gdp = newyork_gdp.rename(columns={'Counties': 'County'})

newyork_gdp

,County,2023 GDP
0,New York,1791210722
1,Albany,35778797
2,Allegany,1431876
3,Bronx,44029337
4,Broome,9026669
...,...,...
58,Washington,1949798
59,Wayne,3806726
60,Westchester,90241804
61,Wyoming,1760078


In [ ]:
newyorkcharge = pd.read_csv('/content/ny_charging.csv', delimiter=';')

# Convert relevant columns to integers, handling potential errors
newyorkcharge['EV Level1 EVSE Num'] = pd.to_numeric(newyorkcharge['EV Level1 EVSE Num'], errors='coerce').fillna(0).astype(int)
newyorkcharge['EV Level2 EVSE Num'] = pd.to_numeric(newyorkcharge['EV Level2 EVSE Num'], errors='coerce').fillna(0).astype(int)
newyorkcharge['EV DC Fast Count'] = pd.to_numeric(newyorkcharge['EV DC Fast Count'], errors='coerce').fillna(0).astype(int)
newyorkcharge['EV Other Info'] = pd.to_numeric(newyorkcharge['EV Other Info'], errors='coerce').fillna(0).astype(int)

# Calculate total and create new column
newyorkcharge['Total'] = newyorkcharge['EV Level1 EVSE Num'] + newyorkcharge['EV Level2 EVSE Num'] + newyorkcharge['EV DC Fast Count'] + newyorkcharge['EV Other Info']

newyorkcharge

,City,State,ZIP,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,EV Other Info,Total
0,Old Westbury,NY,11568,4,0,0,0,4
1,Albany,NY,12203,1,7,0,0,8
2,Malta,NY,12020,2,0,0,0,2
3,Malta,NY,12020,1,0,0,0,1
4,Bronx,NY,10468,0,2,0,0,2
...,...,...,...,...,...,...,...,...
4738,Rochester,NY,14618,0,2,0,0,2
4739,Massapequa,NY,11758,0,0,2,0,2
4740,Latham,NY,12110,0,0,6,0,6
4741,Yonkers,NY,10710,0,0,4,0,4


In [ ]:
# Group the dataframe by 'City' and aggregate the specified columns.
newyorkcharge = newyorkcharge.groupby('City').agg({
    'EV Level1 EVSE Num': 'sum',
    'EV Level2 EVSE Num': 'sum',
    'EV DC Fast Count': 'sum',
    'EV Other Info': 'sum',  # Assuming summing other info is appropriate
    'Total': 'sum'
}).reset_index()

# Display the grouped data.
newyorkcharge

,City,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,EV Other Info,Total
0,Akwesasne,0,4,0,0,4
1,Albany,1,570,28,0,599
2,Albion,0,6,0,0,6
3,Alexander,0,4,0,0,4
4,Alexandria Bay,0,4,0,0,4
...,...,...,...,...,...,...
697,Yorktown Heights,0,9,14,0,23
698,Yorkville,0,11,5,0,16
699,Yulan,0,4,0,0,4
700,irvington,0,2,0,0,2


In [ ]:
newyorkcities = pd.read_csv('/content/NY_Municipalities_and_County_FIPS_codes_20241216.csv', delimiter=',')
newyorkcities

,County Name,City Name,Town Name,Village Name,Municipality,County FIPS
0,Albany,Albany,NaN,NaN,Albany,36001
1,Montgomery,Amsterdam,NaN,NaN,Amsterdam,36057
2,Cayuga,Auburn,NaN,NaN,Auburn,36011
3,Genesee,Batavia,NaN,NaN,Batavia,36037
4,Dutchess,Beacon,NaN,NaN,Beacon,36027
...,...,...,...,...,...,...
1600,Albany,NaN,Guilderland,NaN,Guilderland,36001
1601,Dutchess,NaN,Rhinebeck,NaN,Rhinebeck,36027
1602,Dutchess,NaN,Rhinebeck,Rhinebeck,Rhinebeck,36027
1603,Saratoga,NaN,Corinth,Corinth,Corinth,36091


In [ ]:
# Clean the 'Location' and relevant columns for matching
newyorkcharge['City'] = newyorkcharge['City'].str.strip().str.lower()
newyorkcities['City Name'] = newyorkcities['City Name'].str.strip().str.lower()
newyorkcities['Town Name'] = newyorkcities['Town Name'].str.strip().str.lower()
newyorkcities['Village Name'] = newyorkcities['Village Name'].str.strip().str.lower()
newyorkcities['Municipality'] = newyorkcities['Municipality'].str.strip().str.lower()

# Function to find the county for a given location
def find_county(location):
    match = newyorkcities[
        (newyorkcities['City Name'] == location) |
        (newyorkcities['Town Name'] == location) |
        (newyorkcities['Village Name'] == location) |
        (newyorkcities['Municipality'] == location)
    ]
    if not match.empty:
        return match['County Name'].values[0]  # Return the first match
    return None  # Return None if no match is found

# Apply the function to the 'Location' column in newyorkchargers
newyorkcharge['County Name'] = newyorkcharge['City'].apply(find_county)

newyorkcharge.loc[426] = ["new york", 0, 1999, 68, 0 , 2067, "New York"]
newyorkcharge.loc[369] = ["manhattan", 0, 53, 0, 0 , 0, "New York"]
newyorkcharge.loc[427] = ["manhattan", 0, 53, 0, 0 , 0, "New York"]
newyorkcharge.loc[336] = ["latham", 0, 165, 12, 0 , 177, "Albany"]
newyorkcharge.loc[204] = ["forest hills", 0, 144, 0, 0 , 144, "Allegheny"]
newyorkcharge.loc[356] = ["long island", 0, 112, 0, 0 , 112, "Queens"]

newyorkcharge = newyorkcharge.dropna()

newyorkcharge

,City,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,EV Other Info,Total,County Name
1,albany,1,570,28,0,599,Albany
2,albion,0,6,0,0,6,Orleans
3,alexander,0,4,0,0,4,Genesee
4,alexandria bay,0,4,0,0,4,Jefferson
5,altamont,0,25,1,0,26,Albany
...,...,...,...,...,...,...,...
693,wurtsboro,0,6,0,0,6,Sullivan
696,yonkers,0,137,31,0,168,Westchester
698,yorkville,0,11,5,0,16,Oneida
700,irvington,0,2,0,0,2,Westchester


In [ ]:
# Group by "County Name" and sum the "Total" column
newyorkcharge = newyorkcharge.groupby('County Name')['Total'].sum()

# Convert the Series to a DataFrame with the desired column name
newyorkcharge = newyorkcharge.to_frame(name='Total Amount') # Changed here

# Rename the index to 'County'
newyorkcharge = newyorkcharge.rename_axis('County')  # Changed here



# Display the result
newyorkcharge

,Total Amount
County,
Albany,1042
Allegany,5
Allegheny,144
Bronx,127
Broome,197
...,...
Washington,6
Wayne,63
Westchester,872


In [ ]:
newyorkev = pd.read_csv('/content/ny_ev.csv', delimiter=',')

newyorkev = newyorkev.drop(columns=['Record Type', 'Fuel Type'])

newyorkev.rename(columns={'Registration Class': 'Total EVs'}, inplace=True)

newyorkev['County'] = newyorkev['County'].str.lower()

newyorkev['County'] = newyorkev['County'].str.capitalize()

newyorkev.loc[30] = ["New York", 7210]

# Assuming you want to remove the row with index 2
newyorkev = newyorkev.drop(index=39)

newyorkev

,County,Total EVs
0,Albany,3084
1,Allegany,85
2,Bronx,4194
3,Broome,842
4,Cattaraugus,127
...,...,...
58,Washington,213
59,Wayne,481
60,Westchester,17272
61,Wyoming,77


In [ ]:
# newyork_population
# newyorkcharge
# newyork_gdp
# newyorkev

# First, merge county_counts with df_countyGDP
merged_df = pd.merge(newyork_gdp, newyorkev, on='County', how='left')

# Then merge the result with df_washington
merged_df['County'] = merged_df['County'].str.strip().str.lower()
newyork_population['County'] = newyork_population['County'].str.strip().str.lower()

merged_df = pd.merge(merged_df, newyork_population, on='County', how='left')

# Then merge the result with df_washington
newyorkcharge = newyorkcharge.reset_index()
newyorkcharge['County'] = newyorkcharge['County'].str.strip().str.lower()
merged_df = pd.merge(merged_df, newyorkcharge, on='County', how='left')

ML_TABLE = merged_df[['County', '2023 GDP', 'Total EVs', 'pop2024', 'landArea', 'density', 'Total Amount']]

ML_TABLE = ML_TABLE.rename(columns={'Total Amount': 'Total Chargers'})

ML_TABLE = ML_TABLE.rename(columns={'pop2024': 'Population'})

ML_TABLE = ML_TABLE.rename(columns={'landArea': 'Land Area'})

ML_TABLE = ML_TABLE.rename(columns={'density': 'Population Density'})

ML_TABLE = ML_TABLE.rename(columns={'Total EVs': 'Number of Vehicles'})

ML_TABLE = ML_TABLE.sort_values('Number of Vehicles', ascending=False)

ML_TABLE = ML_TABLE.drop(index=31)

# Capitalize the first letter of each word in a specific column
ML_TABLE['County'] = ML_TABLE['County'].str.title()

ML_TABLE

,County,2023 GDP,Number of Vehicles,Population,Land Area,Population Density,Total Chargers
30,Nassau,100089673,24787.0,1378136,285,4835,262.0
52,Suffolk,105751656,19654.0,1518366,911,1666,328.0
60,Westchester,90241804,17272.0,989898,431,2296,872.0
41,Queens,107263194,16850.0,2225834,109,20420,702.0
24,Kings,110580419,11373.0,2532919,69,36708,717.0
...,...,...,...,...,...,...,...
49,Schuyler,538245,82.0,17405,328,53,26.0
61,Wyoming,1760078,77.0,39463,593,66,16.0
25,Lewis,1219155,49.0,26501,1276,20,8.0
21,Hamilton,235759,31.0,5043,1717,2,22.0


In [ ]:
texas_population = pd.read_csv('/content/texaspop.csv')

texas_population = texas_population.drop(columns=['name', 'pop2020', 'year', 'fips', 'state', 'stateCode', 'growthSince2020', 'slug', 'rank', 'densityMi', 'areaMi'])

texas_population['county'] = texas_population['county'].str.replace(' County', '', regex=False)

texas_population['pop2024'] = texas_population['pop2024'].astype(int)

texas_population['landArea'] = texas_population['landArea'].astype(int)

texas_population['density'] = texas_population['density'].astype(int)

# Renaming the 'county' column in df_washington to match 'County'
texas_population = texas_population.rename(columns={'county': 'County'})

texas_population


,pop2024,County,landArea,density
0,4888913,Harris,1707,2864
1,2610723,Dallas,873,2990
2,2210248,Tarrant,865,2555
3,2115167,Bexar,1240,1705
4,1342372,Travis,994,1350
...,...,...,...,...
249,567,McMullen,1140,0
250,564,Borden,897,0
251,332,Kenedy,1459,0
252,200,King,911,0


In [ ]:
texas_gdp = pd.read_csv('/content/texas_gdp.csv', delimiter=';')

texas_gdp = texas_gdp.drop(columns=['2020 GDP', '2021 GDP', '2022 GDP', 'Rank in State', '2020 Percent Change', '2021 Percent Change', '2022 Percent Change', '2023 Percent Change'])

texas_gdp['2023 GDP'] = texas_gdp['2023 GDP'].str.replace('.', '', regex=False)

# Renaming the 'county' column in df_washington to match 'County'
texas_gdp = texas_gdp.rename(columns={'Counties': 'County'})

texas_gdp = texas_gdp.drop(index=0)

texas_gdp['2023 GDP'] = texas_gdp['2023 GDP'].astype(int)

texas_gdp

,County,2023 GDP
1,Anderson,2164042
2,Andrews,4128580
3,Angelina,3512511
4,Aransas,786741
5,Archer,340087
...,...,...
250,Wood,1582826
251,Yoakum,1983345
252,Young,1152963
253,Zapata,586267


In [ ]:
# Example: Select rows by index
texas_gdp_selected_rows = texas_gdp.loc[[1, 2, 3, 4, 5, 250, 251, 252, 253, 254]]
texas_gdp_selected_rows = texas_gdp_selected_rows.to_latex(index=False, escape=False, column_format='||c c||', caption='2023 Texas County GDP')
print(texas_gdp_selected_rows)

\begin{table}
\caption{2023 Texas County GDP}
\begin{tabular}{||c c||}
\toprule
County & 2023 GDP \\
\midrule
Anderson & 2164042 \\
Andrews & 4128580 \\
Angelina & 3512511 \\
Aransas & 786741 \\
Archer & 340087 \\
Wood & 1582826 \\
Yoakum & 1983345 \\
Young & 1152963 \\
Zapata & 586267 \\
Zavala & 804738 \\
\bottomrule
\end{tabular}
\end{table}



In [ ]:
texas_ev = pd.read_csv('/content/texas_ev.csv', delimiter=';')

texas_ev = texas_ev.drop(columns=['All Vehicles', 'Ratio of EVs to Total Vehicles'])

texas_ev['EV Count'] = texas_ev['EV Count'].str.replace(',', '', regex=False)

texas_ev['EV Count'] = pd.to_numeric(texas_ev['EV Count'], errors='coerce').fillna(0).astype(int)

texas_ev

,County,EV Count
0,Harris,47604
1,Travis,41314
2,Dallas,33195
3,Collin,32771
4,Bexar,23403
...,...,...
248,Hall,0
249,McMullen,0
250,Kenedy,0
251,Roberts,0


In [ ]:
texascharge = pd.read_csv('/content/texas_charging.csv', delimiter=';')

# Convert relevant columns to integers, handling potential errors
texascharge['EV Level1 EVSE Num'] = pd.to_numeric(texascharge['EV Level1 EVSE Num'], errors='coerce').fillna(0).astype(int)
texascharge['EV Level2 EVSE Num'] = pd.to_numeric(texascharge['EV Level2 EVSE Num'], errors='coerce').fillna(0).astype(int)
texascharge['EV DC Fast Count'] = pd.to_numeric(texascharge['EV DC Fast Count'], errors='coerce').fillna(0).astype(int)

# Calculate total and create new column
texascharge['Total'] = texascharge['EV Level1 EVSE Num'] + texascharge['EV Level2 EVSE Num'] + texascharge['EV DC Fast Count']

texascharge

,City,State,ZIP,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,Total
0,Houston,TX,77002,0,2,0,2
1,Austin,TX,78745,0,2,1,3
2,Abilene,TX,79606,0,1,0,1
3,Amarillo,TX,79119,0,2,0,2
4,Arlington,TX,76018,0,1,0,1
...,...,...,...,...,...,...,...
3885,San Antonio,TX,78250,0,0,4,4
3886,Dallas,TX,75240,0,0,4,4
3887,Balcones Heights,TX,78201,0,0,6,6
3888,Houston,TX,77084,0,0,6,6


In [ ]:
# Group the dataframe by 'City' and aggregate the specified columns.
texascharge = texascharge.groupby('City').agg({
    'EV Level1 EVSE Num': 'sum',
    'EV Level2 EVSE Num': 'sum',
    'EV DC Fast Count': 'sum',
    'Total': 'sum'
}).reset_index()

# Display the grouped data.
texascharge

,City,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,Total
0,Abbott,0,0,8,8
1,Abilene,0,7,0,7
2,Addison,0,28,0,28
3,Alamo,0,4,0,4
4,Alamo Heights,0,3,0,3
...,...,...,...,...,...
419,Wimberley,0,4,1,5
420,Winnie,0,6,0,6
421,Woodway,0,2,0,2
422,Wylie,0,2,0,2


In [ ]:
texascities = pd.read_csv('/content/texas_cities.csv', delimiter=';')

texascities

,COUNTY,CITY
0,ANDERSON,CAYUGA
1,ANDERSON,ELKHART
2,ANDERSON,FRANKSTON
3,ANDERSON,MONTALBA
4,ANDERSON,PALESTINE
...,...,...
1389,YOUNG,OLNEY
1390,ZAPATA,ZAPATA
1391,ZAVALA,BATESVILLE
1392,ZAVALA,CRYSTAL CITY


In [ ]:
# Clean the 'Location' and relevant columns for matching
texascities['CITY'] = texascities['CITY'].str.strip().str.lower()
texascities['COUNTY'] = texascities['COUNTY'].str.strip().str.lower()
texascharge['City'] = texascharge['City'].str.strip().str.lower()


# Function to find the county for a given location
def find_county(location):
    match = texascities[
        (texascities['CITY'] == location) |
        (texascities['COUNTY'] == location)
    ]
    if not match.empty:
        return match['COUNTY'].values[0]  # Return the first match
    return None  # Return None if no match is found

# Apply the function to the 'Location' column in newyorkchargers
texascharge['County Name'] = texascharge['City'].apply(find_county)

texascharge.loc[306] = ["plano", 0, 286, 39, 323 , "collin"]

texascharge

,City,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,Total,County Name
0,abbott,0,0,8,8,hill
1,abilene,0,7,0,7,taylor
2,addison,0,28,0,28,dallas
3,alamo,0,4,0,4,hidalgo
4,alamo heights,0,3,0,3,bexar
...,...,...,...,...,...,...
419,wimberley,0,4,1,5,hays
420,winnie,0,6,0,6,chambers
421,woodway,0,2,0,2,None
422,wylie,0,2,0,2,None


In [ ]:
# Group by "County Name" and sum the "Total" column
texascharge = texascharge.groupby('County Name')['Total'].sum()

# Convert the Series to a DataFrame with the desired column name
texascharge = texascharge.to_frame(name='Total Amount') # Changed here

# Rename the index to 'County'
texascharge = texascharge.rename_axis('County')  # Changed here



# Display the result
texascharge

,Total Amount
County,
anderson,6
angelina,7
aransas,27
atascosa,22
austin,1582
...,...
willacy,2
williamson,267
wilson,3


In [ ]:
# texas_population
# texascharge
# texas_gdp
# texas_ev

merged_df = pd.merge(texas_gdp, texas_ev, on='County', how='left')

# Then merge the result with df_washington
merged_df['County'] = merged_df['County'].str.strip().str.lower()
texas_population['County'] = texas_population['County'].str.strip().str.lower()

merged_df = pd.merge(merged_df, texas_population, on='County', how='left')

# Then merge the result with df_washington
texascharge = texascharge.reset_index()
texascharge['County'] = texascharge['County'].str.strip().str.lower()
merged_df = pd.merge(merged_df, texascharge, on='County', how='left')

ML_TABLE2 = merged_df[['County', '2023 GDP', 'EV Count', 'pop2024', 'landArea', 'density', 'Total Amount']]

ML_TABLE2 = ML_TABLE2.rename(columns={'Total Amount': 'Total Chargers'})

ML_TABLE2 = ML_TABLE2.rename(columns={'pop2024': 'Population'})

ML_TABLE2 = ML_TABLE2.rename(columns={'landArea': 'Land Area'})

ML_TABLE2 = ML_TABLE2.rename(columns={'density': 'Population Density'})

ML_TABLE2 = ML_TABLE2.rename(columns={'EV Count': 'Number of Vehicles'})

ML_TABLE2 = ML_TABLE2.sort_values('Number of Vehicles', ascending=False)

# Capitalize the first letter of each word in a specific column
ML_TABLE2['County'] = ML_TABLE2['County'].str.title()

ML_TABLE2.fillna(0, inplace=True)

ML_TABLE2.loc[226] = ["Travis", 159612559	, 41314.0, 1342372, 994, 1350, 402]

ML_TABLE2

,County,2023 GDP,Number of Vehicles,Population,Land Area,Population Density,Total Chargers
100,Harris,429677330,47604.0,4888913,1707,2864,1822.0
226,Travis,159612559,41314.0,1342372,994,1350,402.0
56,Dallas,304515038,33195.0,2610723,873,2990,1304.0
42,Collin,87862787,32771.0,1231723,841,1464,323.0
14,Bexar,121391325,23403.0,2115167,1240,1705,669.0
...,...,...,...,...,...,...,...
54,Culberson,2996840,0.0,2229,3812,0,16.0
61,Dewitt,3451192,0.0,20104,909,22,5.0
82,Gaines,1998582,0.0,22906,1502,15,0.0
141,La Salle,3137796,0.0,6518,1487,4,15.0


In [ ]:
ML_TABLE = pd.concat([ML_TABLE, ML_TABLE2], ignore_index=True)

ML_TABLE = ML_TABLE.sort_values('Number of Vehicles', ascending=False)

# Drop duplicate rows based on all columns
ML_TABLE.drop_duplicates(inplace=True)
# Reset the index of the DataFrame after removing duplicates
ML_TABLE.reset_index(drop=True, inplace=True)

ML_TABLE

,County,2023 GDP,Number of Vehicles,Population,Land Area,Population Density,Total Chargers
0,Harris,429677330,47604.0,4888913,1707,2864,1822.0
1,Travis,159612559,41314.0,1342372,994,1350,402.0
2,Dallas,304515038,33195.0,2610723,873,2990,1304.0
3,Collin,87862787,32771.0,1231723,841,1464,323.0
4,Nassau,100089673,24787.0,1378136,285,4835,262.0
...,...,...,...,...,...,...,...
311,Cottle,109044,0.0,1271,901,1,0.0
312,Terrell,56969,0.0,686,2358,0,0.0
313,Mcmullen,1544293,0.0,567,1140,0,0.0
314,Throckmorton,160434,0.0,1513,913,1,0.0


In [ ]:
# Save the DataFrame to a CSV file

ML_TABLE.to_csv('ML_TABLE2.csv', index=False)


# Download the CSV file

from google.colab import files

files.download('ML_TABLE2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>